# Import Library

In [1]:
import numpy as np
import pandas as pd
import pickle

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# Import Data

In [2]:
meta = pd.read_csv('movies_metadata.csv')
meta.head()

c:\Users\abiza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173.0


In [3]:
links_small = pd.read_csv('links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


# Preprocessing

In [4]:
meta = meta[['id', 'imdb_id', 'title', 'overview', 'tagline', 'genres']].dropna().drop_duplicates(subset='title', keep='first')
meta['id'] = meta['id'].astype('int')
meta['genres'] = meta['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
meta = meta.sort_values('title')

meta.head()

NumExpr defaulting to 4 threads.


,id,imdb_id,title,overview,tagline,genres
18757,55245,tt1699720,!Women Art Revolution,"Through intimate interviews, provocative art, and rare, historical film and video footage, this feature documentary reveals how art addressing political consequences of discrimination and violence, the Feminist Art Revolution radically transformed the art and culture of our times.",40 years in the making!,[Documentary]
30961,41371,tt1637976,#1 Cheerleader Camp,A pair of horny college guys get summer jobs at a sexy cheerleader camp.,Grab your spirit sticks!,"[Comedy, Drama]"
36153,301325,tt3526286,#Horror,"Inspired by actual events, a group of 12 year old girls face a night of horror when the compulsive addiction of an online social media game turns a moment of cyber bullying into a night of insanity.",Death is trending.,"[Drama, Mystery, Horror, Thriller]"
23501,267752,tt3060338,#chicagoGirl,"From her childhood bedroom in the Chicago suburbs, an American teenage girl uses social media to run the revolution in Syria. Armed with Facebook, Twitter, Skype and cameraphones, she helps her social network in Damascus and Homs braves snipers and shelling in the streets and the world the human rights atrocities of one of the most brutal dictators. But as the revolution rages on, everyone in the network must decide what is the most effective way to fight a dictator: social media or AK-47s.",The Social Network Takes on a Dictator,[Documentary]
15775,4204,tt1024733,$5 a Day,"After being released from jail, the son of a con man joins his father on the road.",Relationships don't come cheap!,"[Drama, Comedy]"


In [5]:
links_small = links_small[['tmdbId']].dropna()
links_small = links_small['tmdbId'].astype('int')

links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int32

In [6]:
smd = meta[meta['id'].isin(links_small)]

smd.head()

,id,imdb_id,title,overview,tagline,genres
13938,53830,tt0024805,'Neath the Arizona Skies,"Chris Morrell, the guardian of half-Indian girl Nina, is helping her find her missing white father. so she can cash in on her late mother's oil lease. Outlaw Sam Black is after the girl and her father as well. Besides dealing with the Black gang, Morrell has to find another robber, Jim Moore, who switches clothes with him after he finds Chris unconscious from a fight with Sam Black. Along the way, he meets a lady who's the sister of Jim Moore, another bad hombre who's in cahoots with Jim Moore, and an old friend who takes in Nina and helps Chris locate Nina's father and fight off the various desperadoes",Hair-trigger Action! Hair-Trigger Action! (reissue poster),"[Action, Western]"
13874,19913,tt1022603,(500) Days of Summer,"Tom (Joseph Gordon-Levitt), greeting-card writer and hopeless romantic, is caught completely off-guard when his girlfriend, Summer (Zooey Deschanel), suddenly dumps him. He reflects on their 500 days together to try to figure out where their love affair went sour, and in doing so, Tom rediscovers his true passions in life.",It was almost like falling in love.,"[Comedy, Drama, Romance]"
3721,8420,tt0049189,...And God Created Woman,"Juliette Hardy is sexual dynamite, and has the men of a French coastal town panting. But Antoine, the only man who affects her likewise, wouldn't dream of settling down with a woman his friends consider the town tramp.",...but the devil invented Brigitte Bardot!,"[Drama, Romance]"
6509,9051,tt0078721,10,A Hollywood songwriter goes through a mid-life crisis and becomes infatuated with a sexy blonde newlywed.,"The funniest, sexiest midlife crisis in movie history.","[Comedy, Romance]"
36251,333371,tt1179933,10 Cloverfield Lane,"After a car accident, Michelle awakens to find herself in a mysterious bunker with two men named Howard and Emmett. Howard offers her a pair of crutches to help her remain mobile with her leg injury sustained from the car crash and tells her to ""get good on those"" before leaving the bunker. She has been given the information that there has been an alien attack and the outside world is poisoned. However, Howard and Emmett's intentions soon become questionable and Michelle is faced with a question: Is it better in here or out there?",Monsters come in many forms.,"[Thriller, Science Fiction, Drama]"


# Feature Extraction

In [7]:
smd['description'] = smd['overview'] + smd['tagline']

<ipython-input-7-362f3d9a1c27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']


In [8]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [9]:
tfidf_matrix.shape

(6791, 211931)

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]

array([1.        , 0.        , 0.        , ..., 0.        , 0.00280284,
       0.        ])

In [11]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

tmdb_id = smd['id']
indices_tmdb_id = pd.Series(smd.index, index=smd['id'])

# Resulted Model

In [12]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:16]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

def get_recommendations_id(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:16]
    movie_indices = [i[0] for i in sim_scores]
    return tmdb_id.iloc[movie_indices]

In [13]:
get_recommendations('Jumanji')

3743                        Pixels
2113       Guardians of the Galaxy
4566                    Stay Alive
3591                         Ouija
6711                Wreck-It Ralph
1550            Dungeons & Dragons
1172               Comfort and Joy
6709              Would You Rather
2081                 Grandma's Boy
5265     The Giant Spider Invasion
3703                     Peter Pan
4371             Sleeping Dogs Lie
6765    Zathura: A Space Adventure
6785                      eXistenZ
3880                    Quarantine
Name: title, dtype: object

In [14]:
get_recommendations_id('Jumanji')

3743    257344
2113    118340
4566     10069
3591    242512
6711     82690
1550     11849
1172     30278
6709     97051
2081      9900
5265     39154
3703     10693
4371     43673
6765      6795
6785      1946
3880     13812
Name: id, dtype: int32

In [15]:
movie_list = smd[['id', 'title', 'description', 'genres']]
movie_list.columns = ['movie_id', 'title', 'desc', 'genres']

movie_list.head()

,movie_id,title,desc,genres
0,53830,'Neath the Arizona Skies,"Chris Morrell, the guardian of half-Indian girl Nina, is helping her find her missing white father. so she can cash in on her late mother's oil lease. Outlaw Sam Black is after the girl and her father as well. Besides dealing with the Black gang, Morrell has to find another robber, Jim Moore, who switches clothes with him after he finds Chris unconscious from a fight with Sam Black. Along the way, he meets a lady who's the sister of Jim Moore, another bad hombre who's in cahoots with Jim Moore, and an old friend who takes in Nina and helps Chris locate Nina's father and fight off the various desperadoesHair-trigger Action! Hair-Trigger Action! (reissue poster)","[Action, Western]"
1,19913,(500) Days of Summer,"Tom (Joseph Gordon-Levitt), greeting-card writer and hopeless romantic, is caught completely off-guard when his girlfriend, Summer (Zooey Deschanel), suddenly dumps him. He reflects on their 500 days together to try to figure out where their love affair went sour, and in doing so, Tom rediscovers his true passions in life.It was almost like falling in love.","[Comedy, Drama, Romance]"
2,8420,...And God Created Woman,"Juliette Hardy is sexual dynamite, and has the men of a French coastal town panting. But Antoine, the only man who affects her likewise, wouldn't dream of settling down with a woman his friends consider the town tramp....but the devil invented Brigitte Bardot!","[Drama, Romance]"
3,9051,10,"A Hollywood songwriter goes through a mid-life crisis and becomes infatuated with a sexy blonde newlywed.The funniest, sexiest midlife crisis in movie history.","[Comedy, Romance]"
4,333371,10 Cloverfield Lane,"After a car accident, Michelle awakens to find herself in a mysterious bunker with two men named Howard and Emmett. Howard offers her a pair of crutches to help her remain mobile with her leg injury sustained from the car crash and tells her to ""get good on those"" before leaving the bunker. She has been given the information that there has been an alien attack and the outside world is poisoned. However, Howard and Emmett's intentions soon become questionable and Michelle is faced with a question: Is it better in here or out there?Monsters come in many forms.","[Thriller, Science Fiction, Drama]"


In [16]:
pickle.dump(movie_list, open('movie_list.pkl', 'wb'))

In [17]:
pickle.dump(cosine_sim, open('similarity.pkl', 'wb'))